In [1]:
import pandas as pd
import numpy as np
import requests
from tqdm.auto import tqdm, trange

In [2]:
df_raw = pd.read_csv("서울특별시_전동킥보드_견인_현황_20220731.csv", encoding="cp949", index_col=0)
df_raw

,신고일,구정보,주소,유형,조치일
번호,,,,,
1,2021-07-15,동작구,서울특별시 동작구 상도동 215-45,"점자블록, 엘리베이터 입구",2021-07-15
2,2021-07-15,동작구,서울특별시 동작구 상도동 172-28,보도 중앙,2021-07-15
3,2021-07-15,동작구,서울특별시 동작구 대방동 356-2,"버스정류장, 택시 승강장",2021-07-15
4,2021-07-15,동작구,서울특별시 동작구 대방동 354-1,"버스정류장, 택시 승강장",2021-07-15
5,2021-07-15,동작구,서울특별시 동작구 신대방동 460-30,보도와 차도가 구분된 도로의 차도,2021-07-15
...,...,...,...,...,...
60601,2022-07-29,동작구,서울특별시 동작구 사당동 708-1025,보도와 차도가 구분된 도로의 차도,2022-07-29
60602,2022-07-29,관악구,서울특별시 관악구 봉천동 1693-55,보도와 차도가 구분된 도로의 차도,2022-07-29
60603,2022-07-29,동작구,서울특별시 동작구 사당동 175-56,보도와 차도가 구분된 도로의 차도,2022-07-29


In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60605 entries, 1 to 60605
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   신고일     60605 non-null  object
 1   구정보     60605 non-null  object
 2   주소      60605 non-null  object
 3   유형      60604 non-null  object
 4   조치일     60604 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [4]:
df_raw['날짜'] = pd.to_datetime(df_raw['신고일'])

In [5]:
df = df_raw[['날짜', '주소']]

In [6]:
# 날짜로부터 월, 주말 여부 컬럼 생성
df.insert(1, '월', df['날짜'].dt.month)
df.insert(2, "주말여부", df['날짜'].dt.dayofweek > 4)
df['주말여부'] = df['주말여부'].apply(lambda x: '주말' if x==True else '평일')

/var/folders/3m/knb96f9s4xg2t2gcrl96r6jh0000gn/T/ipykernel_38436/3973990059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주말여부'] = df['주말여부'].apply(lambda x: '주말' if x==True else '평일')


In [7]:
locations = []
for addr in tqdm(df['주소']):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK df5989bfdeb66e539382f8c9a0275d74"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)
    
    #d73f478209140e5db9caeb3ccca66181

  0%|          | 0/60605 [00:00<?, ?it/s]

In [8]:
len(locations)

60605

In [9]:
error_list = []
for i in df.index:
    try:
        df.loc[i, 'x']= locations[i][0]['address']['x']
        df.loc[i, 'y']= locations[i][0]['address']['y']
        df.loc[i, '행정구']= locations[i][0]['address']['region_2depth_name']
        df.loc[i, '행정동']= locations[i][0]['address']['region_3depth_h_name']
    except:
        error_list.append(i)

/var/folders/3m/knb96f9s4xg2t2gcrl96r6jh0000gn/T/ipykernel_38436/3072815986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'x']= locations[i][0]['address']['x']


In [10]:
df.to_csv("킥보드 견인정보.csv")
df

,날짜,월,주말여부,주소,x,y,행정구,행정동
번호,,,,,,,,
1,2021-07-15,7,평일,서울특별시 동작구 상도동 215-45,126.941330110996,37.5039201815926,동작구,상도2동
2,2021-07-15,7,평일,서울특별시 동작구 상도동 172-28,126.924511702849,37.5099598878193,동작구,대방동
3,2021-07-15,7,평일,서울특별시 동작구 대방동 356-2,126.92514786773,37.5102350989971,동작구,대방동
4,2021-07-15,7,평일,서울특별시 동작구 대방동 354-1,126.915087569973,37.4937083584454,동작구,신대방2동
5,2021-07-15,7,평일,서울특별시 동작구 신대방동 460-30,126.940490937931,37.5131864148732,동작구,노량진1동
...,...,...,...,...,...,...,...,...
60601,2022-07-29,7,평일,서울특별시 동작구 사당동 708-1025,126.957896785606,37.4797637636311,관악구,행운동
60602,2022-07-29,7,평일,서울특별시 관악구 봉천동 1693-55,126.970701011629,37.4900448947379,동작구,사당3동
60603,2022-07-29,7,평일,서울특별시 동작구 사당동 175-56,126.970577192578,37.4900509007176,동작구,사당3동


In [11]:
#df.reset_index(drop=True, inplace=True)

In [34]:
df

,날짜,월,주말여부,주소,x,y,행정구,행정동
0,2021-07-15,7,평일,서울특별시 동작구 상도동 215-45,126.941330110996,37.5039201815926,동작구,상도2동
1,2021-07-15,7,평일,서울특별시 동작구 상도동 172-28,126.924511702849,37.5099598878193,동작구,대방동
2,2021-07-15,7,평일,서울특별시 동작구 대방동 356-2,126.92514786773,37.5102350989971,동작구,대방동
3,2021-07-15,7,평일,서울특별시 동작구 대방동 354-1,126.915087569973,37.4937083584454,동작구,신대방2동
4,2021-07-15,7,평일,서울특별시 동작구 신대방동 460-30,126.940490937931,37.5131864148732,동작구,노량진1동
...,...,...,...,...,...,...,...,...
60600,2022-07-29,7,평일,서울특별시 동작구 사당동 708-1025,126.957896785606,37.4797637636311,관악구,행운동
60601,2022-07-29,7,평일,서울특별시 관악구 봉천동 1693-55,126.970701011629,37.4900448947379,동작구,사당3동
60602,2022-07-29,7,평일,서울특별시 동작구 사당동 175-56,126.970577192578,37.4900509007176,동작구,사당3동
60603,2022-07-29,7,평일,서울특별시 동작구 사당동 175-26,127.076585278572,37.5840480367436,중랑구,면목5동


In [46]:
for i in error_list:
     print(i, df.loc[i, 'x'])

221 126.913193926769
1452 126.918830677235
1456 126.924373377005
2069 126.92514786773
2515 nan
2516 nan
2517 126.919407694974
4917 126.961745761671
5520 127.086752802997
5922 127.04949598662
6754 126.957664300492
7607 126.887858629521
9243 126.982273311095
12653 126.926723074076
13893 126.82530972828
13956 126.942835953538
14584 126.951972945354
14670 126.981704571951
14760 126.925659595074
14991 126.893054879965
15163 126.816236990628
15263 127.089251085975
15309 126.918830677235
15457 126.873566242652
16121 127.133206737536
16174 126.887484011874
16301 126.918830677235
16926 nan
16927 126.951444914492
16929 126.951444914492
17128 126.952096689061
17135 126.961206915234
18782 126.931528006724
18792 126.878589854433
18796 127.058060359718
18798 nan
18799 126.943422344426
18801 126.881786704654
19989 126.924243532498
20271 127.074575586641
21646 126.924926304623
21839 126.981808199029
21851 127.053421358065
22312 126.983007103325
22466 127.057331203929
22468 127.115507083582
23222 127.0

In [30]:
d

1